# Regression Fundamentals: Comprehensive Tutorial

This notebook provides an in-depth exploration of regression algorithms, evaluation techniques, and advanced patterns for real-world applications.

## 🎯 Learning Objectives

By the end of this tutorial, you'll master:
- **6+ regression algorithms** with their strengths and use cases
- **Advanced evaluation metrics** beyond simple RMSE
- **Regularization techniques** to prevent overfitting
- **Feature engineering** for regression
- **Model interpretation** and explainability
- **Production deployment** considerations

## 📋 Table of Contents

1. [Data Loading and Exploration](#1-data-loading-and-exploration)
2. [Linear Regression Fundamentals](#2-linear-regression-fundamentals)
3. [Regularization Techniques](#3-regularization-techniques)
4. [Tree-Based Regression](#4-tree-based-regression)
5. [Advanced Evaluation Techniques](#5-advanced-evaluation-techniques)
6. [Feature Engineering for Regression](#6-feature-engineering-for-regression)
7. [Polynomial and Non-linear Regression](#7-polynomial-and-non-linear-regression)
8. [Model Selection and Comparison](#8-model-selection-and-comparison)
9. [Production Considerations](#9-production-considerations)
10. [Summary and Best Practices](#10-summary-and-best-practices)

In [ ]:
# Essential imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_diabetes, make_regression
from sklearn.model_selection import (
    train_test_split, cross_val_score, GridSearchCV,
    learning_curve, validation_curve
)
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, MinMaxScaler
from sklearn.linear_model import (
    LinearRegression, Ridge, Lasso, ElasticNet,
    HuberRegressor, RANSACRegressor
)
from sklearn.ensemble import (
    RandomForestRegressor, GradientBoostingRegressor,
    ExtraTreesRegressor, BaggingRegressor
)
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, r2_score,
    mean_absolute_percentage_error, explained_variance_score
)
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.inspection import permutation_importance
from scipy import stats
import warnings
from datetime import datetime

warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("viridis")

print("✅ All packages imported successfully!")
print(f"📅 Notebook started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 1. Data Loading and Exploration

We'll work with multiple datasets to demonstrate different regression scenarios.

In [ ]:
# Load multiple datasets for comprehensive analysis
print("📊 Loading Regression Datasets")
print("=" * 32)

# Dataset 1: Diabetes Dataset (Medical)
diabetes_data = load_diabetes()
diabetes_X = pd.DataFrame(diabetes_data.data, columns=diabetes_data.feature_names)
diabetes_y = diabetes_data.target

print(f"🏥 Diabetes Dataset:")
print(f"   Shape: {diabetes_X.shape}")
print(f"   Target: Disease progression (continuous)")
print(f"   Target range: [{diabetes_y.min():.1f}, {diabetes_y.max():.1f}]")
print(f"   Target mean: {diabetes_y.mean():.1f} ± {diabetes_y.std():.1f}")

# Dataset 2: Create synthetic housing dataset
print(f"\n🏠 Creating Synthetic Housing Dataset:")
np.random.seed(42)
n_samples = 506

# Generate realistic housing features
house_age = np.random.uniform(5, 100, n_samples)
rooms = np.random.normal(6, 1, n_samples)
crime_rate = np.random.exponential(3, n_samples)
distance_to_city = np.random.uniform(1, 12, n_samples)
tax_rate = np.random.normal(400, 100, n_samples)
pupil_teacher_ratio = np.random.normal(18, 3, n_samples)

# Generate price with realistic relationships
price = (
    rooms * 8 +
    -house_age * 0.2 +
    -crime_rate * 2 +
    -distance_to_city * 1.5 +
    -tax_rate * 0.02 +
    -pupil_teacher_ratio * 0.8 +
    np.random.normal(0, 3, n_samples) + 25
)

housing_X = pd.DataFrame({
    'house_age': house_age,
    'avg_rooms': rooms,
    'crime_rate': crime_rate,
    'distance_to_city': distance_to_city,
    'tax_rate': tax_rate,
    'pupil_teacher_ratio': pupil_teacher_ratio
})
housing_y = price

print(f"   Shape: {housing_X.shape}")
print(f"   Target: Median home value ($1000s)")
print(f"   Target range: [${housing_y.min():.1f}k, ${housing_y.max():.1f}k]")
print(f"   Target mean: ${housing_y.mean():.1f}k ± ${housing_y.std():.1f}k")

# Dataset 3: High-dimensional Synthetic Dataset
highdim_X, highdim_y = make_regression(
    n_samples=800,
    n_features=50,
    n_informative=30,
    noise=0.1,
    random_state=42
)
highdim_X = pd.DataFrame(highdim_X, columns=[f'feature_{i:02d}' for i in range(50)])

print(f"\n🔬 High-Dimensional Synthetic Dataset:")
print(f"   Shape: {highdim_X.shape}")
print(f"   Target: Synthetic continuous variable")
print(f"   Target range: [{highdim_y.min():.1f}, {highdim_y.max():.1f}]")
print(f"   Target mean: {highdim_y.mean():.1f} ± {highdim_y.std():.1f}")

print("\n✅ All datasets loaded successfully!")

In [ ]:
# Comprehensive data exploration
def explore_regression_dataset(X, y, dataset_name):
    """
    Comprehensive exploration of a regression dataset.
    """
    print(f"\n🔍 Exploring {dataset_name} Dataset")
    print("=" * (len(dataset_name) + 20))
    
    # Basic statistics
    print(f"📊 Dataset Shape: {X.shape}")
    print(f"🎯 Target Statistics:")
    print(f"   Mean: {y.mean():.3f}")
    print(f"   Std: {y.std():.3f}")
    print(f"   Min: {y.min():.3f}")
    print(f"   Max: {y.max():.3f}")
    print(f"   Range: {y.max() - y.min():.3f}")
    print(f"   Skewness: {stats.skew(y):.3f}")
    
    # Check for normality of target
    _, p_value = stats.shapiro(y[:100] if len(y) > 100 else y)
    if p_value > 0.05:
        print(f"   ✅ Target appears normally distributed (p={p_value:.3f})")
    else:
        print(f"   ⚠️ Target may not be normally distributed (p={p_value:.3f})")
    
    # Outlier detection
    Q1 = np.percentile(y, 25)
    Q3 = np.percentile(y, 75)
    IQR = Q3 - Q1
    outlier_threshold = 1.5 * IQR
    outliers = np.sum((y < Q1 - outlier_threshold) | (y > Q3 + outlier_threshold))
    outlier_percentage = outliers / len(y) * 100
    
    print(f"   Outliers: {outliers} ({outlier_percentage:.1f}%)")
    
    # Feature statistics
    print(f"\n📈 Feature Statistics:")
    print(f"   Feature count: {X.shape[1]}")
    print(f"   Missing values: {X.isnull().sum().sum()}")
    
    # Feature scaling analysis
    feature_ranges = X.max() - X.min()
    max_range = feature_ranges.max()
    min_range = feature_ranges.min()
    
    if max_range / min_range > 10:
        print(f"   🔧 Feature scaling recommended (range ratio: {max_range/min_range:.1f}:1)")
    else:
        print(f"   ✅ Feature scales are relatively similar")
    
    # Correlation analysis
    correlations = X.corrwith(pd.Series(y))
    strong_correlations = correlations[abs(correlations) > 0.5]
    
    print(f"\n🔗 Target Correlations:")
    print(f"   Strong correlations (|r| > 0.5): {len(strong_correlations)}")
    if len(strong_correlations) > 0:
        best_feature = correlations.abs().idxmax()
        print(f"   Strongest: {best_feature} (r={correlations[best_feature]:.3f})")
    
    return {
        'shape': X.shape,
        'target_mean': y.mean(),
        'target_std': y.std(),
        'target_range': y.max() - y.min(),
        'target_skew': stats.skew(y),
        'outlier_percentage': outlier_percentage,
        'needs_scaling': max_range / min_range > 10,
        'strong_correlations': len(strong_correlations)
    }

# Explore all datasets
diabetes_stats = explore_regression_dataset(diabetes_X, diabetes_y, "Diabetes")
housing_stats = explore_regression_dataset(housing_X, housing_y, "Housing")
highdim_stats = explore_regression_dataset(highdim_X, highdim_y, "High-Dimensional")

## 2. Linear Regression Fundamentals

Let's start with the foundation of regression: linear regression and its assumptions.

In [ ]:
# Linear regression fundamentals with housing dataset
print("📈 Linear Regression Fundamentals")
print("=" * 34)

# Use housing dataset for realistic example
X_lr = housing_X
y_lr = housing_y

# Split and scale data
X_train_lr, X_test_lr, y_train_lr, y_test_lr = train_test_split(
    X_lr, y_lr, test_size=0.2, random_state=42
)

# Scale features
scaler_lr = StandardScaler()
X_train_lr_scaled = scaler_lr.fit_transform(X_train_lr)
X_test_lr_scaled = scaler_lr.transform(X_test_lr)

print(f"📊 Training set: {X_train_lr_scaled.shape}")
print(f"📊 Test set: {X_test_lr_scaled.shape}")
print(f"🎯 Target range: ${y_train_lr.min():.0f}k - ${y_train_lr.max():.0f}k")

# Fit linear regression model
lr_model = LinearRegression()
lr_model.fit(X_train_lr_scaled, y_train_lr)

# Make predictions
y_pred_lr = lr_model.predict(X_test_lr_scaled)
y_pred_train_lr = lr_model.predict(X_train_lr_scaled)

# Calculate comprehensive metrics
mse_test = mean_squared_error(y_test_lr, y_pred_lr)
rmse_test = np.sqrt(mse_test)
mae_test = mean_absolute_error(y_test_lr, y_pred_lr)
r2_test = r2_score(y_test_lr, y_pred_lr)
mape_test = mean_absolute_percentage_error(y_test_lr, y_pred_lr)
explained_var = explained_variance_score(y_test_lr, y_pred_lr)

# Training metrics
mse_train = mean_squared_error(y_train_lr, y_pred_train_lr)
r2_train = r2_score(y_train_lr, y_pred_train_lr)

print(f"\n📊 Linear Regression Performance:")
print(f"   Training R²: {r2_train:.3f}")
print(f"   Test R²: {r2_test:.3f}")
print(f"   Test RMSE: ${rmse_test:.1f}k")
print(f"   Test MAE: ${mae_test:.1f}k")
print(f"   Test MAPE: {mape_test:.1%}")
print(f"   Explained Variance: {explained_var:.3f}")

# Analyze overfitting
overfitting_gap = r2_train - r2_test
if overfitting_gap > 0.1:
    print(f"   ⚠️ Potential overfitting detected (gap: {overfitting_gap:.3f})")
elif overfitting_gap < 0.05:
    print(f"   ✅ Good generalization (gap: {overfitting_gap:.3f})")
else:
    print(f"   ⚡ Moderate overfitting (gap: {overfitting_gap:.3f})")

# Analyze coefficients
coefficients = pd.DataFrame({
    'Feature': X_lr.columns,
    'Coefficient': lr_model.coef_,
    'Abs_Coefficient': np.abs(lr_model.coef_)
})
coefficients = coefficients.sort_values('Abs_Coefficient', ascending=False)

print(f"\n🔍 Feature Coefficients (Standardized):")
print(f"   Intercept: ${lr_model.intercept_:.1f}k")
for _, row in coefficients.head(5).iterrows():
    sign = '+' if row['Coefficient'] > 0 else ''
    print(f"   {row['Feature']:15}: {sign}{row['Coefficient']:.2f} (impact: ${abs(row['Coefficient']):.2f}k)")

# Cross-validation analysis
print(f"\n🔄 Cross-Validation Analysis:")
cv_scores = cross_val_score(lr_model, X_train_lr_scaled, y_train_lr, cv=5, scoring='r2')
cv_rmse_scores = cross_val_score(lr_model, X_train_lr_scaled, y_train_lr, cv=5, 
                                scoring='neg_root_mean_squared_error')

print(f"   CV R²: {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")
print(f"   CV RMSE: ${-cv_rmse_scores.mean():.1f}k ± ${cv_rmse_scores.std():.1f}k")

print("\n✅ Linear regression analysis complete!")

## 3. Regularization Techniques

Let's explore Ridge, Lasso, and Elastic Net regression to handle overfitting and feature selection.

In [ ]:
# Regularization techniques comparison
print("🎯 Regularization Techniques")
print("=" * 28)

# Use high-dimensional dataset for regularization demo
X_reg = highdim_X
y_reg = highdim_y

# Split and scale data
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

scaler_reg = StandardScaler()
X_train_reg_scaled = scaler_reg.fit_transform(X_train_reg)
X_test_reg_scaled = scaler_reg.transform(X_test_reg)

print(f"📊 Training set: {X_train_reg_scaled.shape}")
print(f"📊 Test set: {X_test_reg_scaled.shape}")
print(f"🎯 Features: {X_reg.shape[1]} (high-dimensional for regularization demo)")

# Define regularization models with different alpha values
regularization_models = {
    'Linear Regression': LinearRegression(),
    'Ridge (α=0.1)': Ridge(alpha=0.1, random_state=42),
    'Ridge (α=1.0)': Ridge(alpha=1.0, random_state=42),
    'Ridge (α=10.0)': Ridge(alpha=10.0, random_state=42),
    'Lasso (α=0.1)': Lasso(alpha=0.1, random_state=42, max_iter=2000),
    'Lasso (α=1.0)': Lasso(alpha=1.0, random_state=42, max_iter=2000),
    'Elastic Net (α=0.1)': ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42, max_iter=2000),
    'Elastic Net (α=1.0)': ElasticNet(alpha=1.0, l1_ratio=0.5, random_state=42, max_iter=2000)
}

print(f"\n🤖 Training {len(regularization_models)} regularized models...")

regularization_results = {}

for name, model in regularization_models.items():
    print(f"\n🔄 Training {name}...")
    
    start_time = datetime.now()
    
    # Train the model
    model.fit(X_train_reg_scaled, y_train_reg)
    
    # Make predictions
    y_pred_reg = model.predict(X_test_reg_scaled)
    y_pred_train_reg = model.predict(X_train_reg_scaled)
    
    training_time = (datetime.now() - start_time).total_seconds()
    
    # Calculate metrics
    r2_test = r2_score(y_test_reg, y_pred_reg)
    r2_train = r2_score(y_train_reg, y_pred_train_reg)
    rmse_test = np.sqrt(mean_squared_error(y_test_reg, y_pred_reg))
    mae_test = mean_absolute_error(y_test_reg, y_pred_reg)
    
    # Cross-validation
    cv_scores = cross_val_score(model, X_train_reg_scaled, y_train_reg, cv=5, scoring='r2')
    
    # Feature selection analysis (for Lasso and Elastic Net)
    n_selected_features = np.sum(np.abs(model.coef_) > 1e-5) if hasattr(model, 'coef_') else X_reg.shape[1]
    
    regularization_results[name] = {
        'model': model,
        'r2_test': r2_test,
        'r2_train': r2_train,
        'rmse_test': rmse_test,
        'mae_test': mae_test,
        'cv_r2_mean': cv_scores.mean(),
        'cv_r2_std': cv_scores.std(),
        'overfitting_gap': r2_train - r2_test,
        'n_selected_features': n_selected_features,
        'training_time': training_time,
        'predictions': y_pred_reg
    }
    
    print(f"   ✅ R² Test: {r2_test:.3f} | R² Train: {r2_train:.3f} | Gap: {r2_train - r2_test:.3f}")
    print(f"      RMSE: {rmse_test:.2f} | Features: {n_selected_features}/{X_reg.shape[1]}")
    print(f"      CV R²: {cv_scores.mean():.3f}±{cv_scores.std():.3f} | Time: {training_time:.3f}s")

print("\n🏆 Regularization training complete!")

## 4. Tree-Based Regression

Let's explore decision trees, random forests, and gradient boosting for regression.

In [ ]:
# Tree-based regression methods
print("🌳 Tree-Based Regression Methods")
print("=" * 33)

# Use diabetes dataset for tree methods
X_tree = diabetes_X
y_tree = diabetes_y

# Split data
X_train_tree, X_test_tree, y_train_tree, y_test_tree = train_test_split(
    X_tree, y_tree, test_size=0.2, random_state=42
)

print(f"📊 Training set: {X_train_tree.shape}")
print(f"📊 Test set: {X_test_tree.shape}")
print(f"🎯 Target range: [{y_train_tree.min():.1f}, {y_train_tree.max():.1f}]")

# Define tree-based models
tree_models = {
    'Decision Tree': DecisionTreeRegressor(random_state=42, max_depth=10),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Extra Trees': ExtraTreesRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42)
}

print(f"\n🤖 Training {len(tree_models)} tree-based models...")

tree_results = {}

for name, model in tree_models.items():
    print(f"\n🔄 Training {name}...")
    
    start_time = datetime.now()
    
    # Train the model
    model.fit(X_train_tree, y_train_tree)
    
    # Make predictions
    y_pred_tree = model.predict(X_test_tree)
    y_pred_train_tree = model.predict(X_train_tree)
    
    training_time = (datetime.now() - start_time).total_seconds()
    
    # Calculate metrics
    r2_test = r2_score(y_test_tree, y_pred_tree)
    r2_train = r2_score(y_train_tree, y_pred_train_tree)
    rmse_test = np.sqrt(mean_squared_error(y_test_tree, y_pred_tree))
    mae_test = mean_absolute_error(y_test_tree, y_pred_tree)
    mape_test = mean_absolute_percentage_error(y_test_tree, y_pred_tree)
    
    # Cross-validation
    cv_scores = cross_val_score(model, X_train_tree, y_train_tree, cv=5, scoring='r2')
    
    # Feature importance
    feature_importance = None
    if hasattr(model, 'feature_importances_'):
        feature_importance = model.feature_importances_
    
    tree_results[name] = {
        'model': model,
        'r2_test': r2_test,
        'r2_train': r2_train,
        'rmse_test': rmse_test,
        'mae_test': mae_test,
        'mape_test': mape_test,
        'cv_r2_mean': cv_scores.mean(),
        'cv_r2_std': cv_scores.std(),
        'overfitting_gap': r2_train - r2_test,
        'training_time': training_time,
        'feature_importance': feature_importance,
        'predictions': y_pred_tree
    }
    
    print(f"   ✅ R² Test: {r2_test:.3f} | R² Train: {r2_train:.3f} | Gap: {r2_train - r2_test:.3f}")
    print(f"      RMSE: {rmse_test:.2f} | MAE: {mae_test:.2f} | MAPE: {mape_test:.1%}")
    print(f"      CV R²: {cv_scores.mean():.3f}±{cv_scores.std():.3f} | Time: {training_time:.3f}s")

print("\n🏆 Tree-based models training complete!")

## Summary and Best Practices

This comprehensive regression tutorial covers key machine learning concepts and techniques for building effective regression models.

In [ ]:
# Regression best practices and summary
print("🎓 Regression Fundamentals Summary")
print("=" * 35)

# Complete session statistics
total_models_trained = len(regularization_results) + len(tree_results)
total_datasets_used = 3  # Diabetes, Housing, High-dim
techniques_covered = [
    "Linear Regression", "Ridge Regression", "Lasso Regression", "Elastic Net",
    "Random Forest", "Gradient Boosting", "Decision Trees", "Extra Trees"
]

print(f"📊 Session Statistics:")
print(f"   🤖 Total Models Trained: {total_models_trained}")
print(f"   📁 Datasets Analyzed: {total_datasets_used}")
print(f"   🔧 Techniques Covered: {len(techniques_covered)}")

print(f"\n🏆 Key Performance Insights:")
print("=" * 26)

# Algorithm recommendations by use case
print(f"\n💡 Algorithm Recommendations by Use Case:")
print("=" * 43)

use_cases = {
    "High Interpretability": {
        "Primary": "Linear Regression",
        "Alternative": "Lasso Regression",
        "Reason": "Simple coefficients, clear feature relationships"
    },
    "Feature Selection": {
        "Primary": "Lasso Regression",
        "Alternative": "Elastic Net",
        "Reason": "Automatic feature selection through L1 penalty"
    },
    "High Accuracy": {
        "Primary": "Random Forest",
        "Alternative": "Gradient Boosting",
        "Reason": "Complex patterns, non-linear relationships"
    },
    "Fast Prediction": {
        "Primary": "Linear Regression",
        "Alternative": "Ridge Regression",
        "Reason": "Simple linear transformation"
    },
    "Robust to Outliers": {
        "Primary": "Random Forest",
        "Alternative": "Huber Regression",
        "Reason": "Tree-based methods and robust loss functions"
    }
}

for use_case, recommendations in use_cases.items():
    print(f"\n🎯 {use_case}:")
    print(f"   Primary: {recommendations['Primary']}")
    print(f"   Alternative: {recommendations['Alternative']}")
    print(f"   Reason: {recommendations['Reason']}")

# Key metrics reference
print(f"\n📊 Key Metrics Reference:")
print("-" * 25)

metrics_guide = {
    "R² Score": "Proportion of variance explained (0-1, higher better)",
    "RMSE": "Root Mean Square Error (same units as target, lower better)",
    "MAE": "Mean Absolute Error (robust to outliers, lower better)",
    "MAPE": "Mean Absolute Percentage Error (scale-independent, lower better)",
    "Cross-validation": "Robust estimate using multiple train/validation splits"
}

for metric, description in metrics_guide.items():
    print(f"   {metric:18}: {description}")

# Common pitfalls to avoid
print(f"\n⚠️ Common Pitfalls to Avoid:")
print("-" * 28)

pitfalls = [
    "Using R² alone without considering other metrics",
    "Not checking linear regression assumptions",
    "Forgetting to scale features for distance-based algorithms",
    "Overfitting by optimizing on test set",
    "Ignoring outliers in model selection",
    "Not considering multicollinearity in linear models",
    "Using complex models when simple ones suffice"
]

for i, pitfall in enumerate(pitfalls, 1):
    print(f"   {i}. {pitfall}")

print("\n🎉 Regression Fundamentals Complete!")
print("\n🚀 Next Steps:")
print("   • Practice with your own datasets")
print("   • Explore advanced ensemble methods")
print("   • Study feature engineering techniques")
print("   • Learn about neural networks for regression")
print("   • Implement model monitoring in production")

print("\n✅ You are now ready for advanced regression challenges!")